In [1]:
!pip install tensorflow

notebook 5.4.0 requires ipykernel, which is not installed.
jupyter 1.0.0 requires ipykernel, which is not installed.
jupyter-console 5.2.0 requires ipykernel, which is not installed.
ipywidgets 7.1.1 requires ipykernel>=4.5.1, which is not installed.


In [2]:
!pip install git+https://github.com/tflearn/tflearn.git

  Cloning https://github.com/tflearn/tflearn.git to c:\users\grant\appdata\local\temp\pip-req-build-m5w0p7ij
  Running setup.py bdist_wheel for tflearn: started
  Running setup.py bdist_wheel for tflearn: finished with status 'done'
  Stored in directory: C:\Users\grant\AppData\Local\Temp\pip-ephem-wheel-cache-jvvh955b\wheels\5a\18\2a\c62b9937b37223da935fb6b2965f02fbc45691e460a08b91b4
Successfully built tflearn


notebook 5.4.0 requires ipykernel, which is not installed.
jupyter 1.0.0 requires ipykernel, which is not installed.
jupyter-console 5.2.0 requires ipykernel, which is not installed.
ipywidgets 7.1.1 requires ipykernel>=4.5.1, which is not installed.


In [5]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random

In [6]:
# restore all of our data structures
import pickle
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

# import our chat-bot intents file
import json
f = open('c:/Users/grant/desktop/AiLawMiniCourseHW/book.txt', 'r')
s = f.read()

intents = json.loads(s)
print(intents)

{'intents': [{'tag': 'greating', 'patterns': ['Hello', 'How are you doing?', 'Good Morning', 'Good Afternoon'], 'responses': ['Hello!', 'Good to see you!'], 'context_set': ''}, {'tag': 'types', 'patterns': ['What types of damages are available for this case?', 'What types of damages are available to the plaintiff?', 'What types of damages are available in this case?'], 'responses': ['Typically damages are calculate in general and special in California', 'California provides special and general damages to plaintiffs']}, {'tag': 'general', 'patterns': ['Can you explain general damages?', 'Explain general damages to the jury?', 'What is the meaning of general damages?'], 'responses': ['General damages are non-economic damages suffered by the plaintiff', 'General damages are defined as non-economic damages']}, {'tag': 'special', 'patterns': ['Can you explain special damages?', 'Explain special damages to the jury?', 'What is the meaning of special damages?'], 'responses': ['Special damages

In [7]:
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

In [8]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [9]:
p = bow("what are the damages?", words)
print (p)
print (classes)

[0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0]
['calculated', 'construction', 'double', 'economic', 'general', 'greating', 'medical', 'multiplier', 'noneconomic', 'pain', 'single', 'special', 'total', 'types']


In [10]:
# load our saved model
model.load('.\model.tflearn')

INFO:tensorflow:Restoring parameters from C:\Users\grant\model.tflearn


In [11]:
# create a data structure to hold user context
context = {}

ERROR_THRESHOLD = 0.25
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        # a random response from the intent
                        return print(random.choice(i['responses']))
            results.pop(0)

In [37]:
#this connects the chatbot to Amazon Alexa, but it doesn't work currently
import logging
import re
import requests
from flask import Flask, request
from flask_ask import Ask, statement, question, session

app = Flask(__name__)
ask = Ask(app,'/')

@app.route('/')
def homepage():
    if session.attributes.get('last_intent') == "QuestionIntent":
        return 'Hello...'

@ask.launch
def start_skill():
    hellomsg = 'My name is damages bot and I am here to present my opinion of the damages in this case, do you have any questions?'
    return question(hellomsg)
    

@ask.intent("QuestionIntent", convert{"query":str})
def question_intent(query):
    # languange = []
    # language.append(request.args.get("query","Question"))
    letters_only = re.sub("[^a-zA-Z]",  # Search for all non-letters
                           " ",          # Replace all non-letters with spaces
                          str(query))
    #end = []
    #end.append(response(letters_only))
    answer = "Yes...".format(response(letters_only))+"anything else?"
    return question(answer)

@ask.intent("EndIntent")
def end_intent(query):
    goodbyemsg = 'Have a nice day.'
    return statement(goodbyemsg)
    should_session_end = True


if __name__ == '__main__':
    app.run(debug=False)


SyntaxError: invalid syntax (<ipython-input-37-92a3f9793b25>, line 21)

In [12]:
response('hello')

Hello!


In [13]:
response('What types of damages are avaialbe in this case?')

California provides special and general damages to plaintiffs


In [14]:
response('what are special damages?')

Special damages are defined as economic damages


In [16]:
response('what are general damages?')

General damages are non-economic damages suffered by the plaintiff


In [17]:
response('Can you define noneconomic damages?')

Noneconomic damages are intangible losses and can incldue pain and suffering, mental anguish and loss of consortium


In [18]:
response('whare are economic damages?')

Some examples of economic damages are costs of nursing care, physical rehabilitation and vocational rehabilitation, but can include all tangible losses


In [19]:
response('what is the cost of a single wide trailer')

A single wide trailor costs around forty thousand dollars new, and ten to twenty thousand dollars used.


In [20]:
response('what is the cost of a double wide trailer')

A double wide trailor costs around seventy five thousand dollars new, and twenty to fifty thousand dollars used.


In [22]:
response('what would be the cost of construction to repair the trailer')

Construction costs for a modular home range from seventeen thousand dollars at the low end up to fifty thousand dollars.


In [24]:
response('how do you calculate pain and suffering')

Usually pain and suffering is calculated multiplying the medical bills by a multiplier between one and five.


In [25]:
response('what are the calculated pain and suffering damages that should be awarded to the plaintiff')

Two thousand dollars.


In [28]:
response('what are the total damages in this case')